In [1]:
from platform import python_version
print(python_version())

3.6.2


In [2]:
from pylds.lds_messages_interface import kalman_filter, E_step
import numpy as np
import numpy.random as npr
import h5py
import sys
import time
import matplotlib.pyplot as plt
from pylds.models import DefaultLDS
sys.path.append('/Users/anastasia/Documents/Github/rand_tensor/')
from utils import *
from randtensor import *

/Users/anastasia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
filename = '/Users/anastasia/Desktop/zimmer/WT_data.mat'
f = h5py.File(filename,'r') 
key0 = list(f.keys())[0]
key1 = list(f.keys())[1]
key2 = list(f.keys())[2]

group0 = f[key0] # key is #refs#, group with 1184 members
group0keys = list(group0.keys()) #(0-9, A-Z, a-z) + (b-t)

group1 = f[key1] # key is #subsystem#, group with 1 member
group1keys = list(group1.keys()) #MCOS dataset

group2 = f[key2] # key is WT_data, group with 5 member
group2keys = list(group2.keys()) 
group2_variables = list(group2['N1'].keys()) #['NeuronNames', 'Opts', 'States', 'dataset', 'deltaFOverF', 'deltaFOverF_bc', 'fps', 'processed', 'tv']

x = np.array(group2['N1']['processed']['deltaFOverF_bc_detr'][:])

In [4]:
######example from EM.py
npr.seed(0)

# Set parameters
D_obs = 1
D_latent = 2
D_input = 0
T = 2000

# Simulate from one LDS
truemodel = DefaultLDS(D_obs, D_latent, D_input)
inputs = np.random.randn(T, D_input)
data, stateseq = truemodel.generate(T, inputs=inputs)

# Fit with another LDS
model = DefaultLDS(D_obs, D_latent, D_input)
model.add_data(data, inputs=inputs)

ImportError: dlopen(/Users/anastasia/anaconda3/lib/python3.6/site-packages/pypolyagamma/pypolyagamma.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libgsl.23.dylib
  Referenced from: /Users/anastasia/anaconda3/lib/python3.6/site-packages/pypolyagamma/pypolyagamma.cpython-36m-darwin.so
  Reason: image not found

In [5]:
T = 3000
D = 30
N = 100

data = npr.randn(T, N)
inputs = np.zeros((T, 0))
mu_init = np.zeros(D)
sigma_init = np.eye(D)
A = np.eye(D)
B  = np.zeros((D, 0))
sigma_states = np.eye(D)
C = npr.randn(N, D)
D = np.zeros((N, 0))
sigma_obs = np.eye(N)

#lls, filtered_mus, filtered_sigmas = \
#    kalman_filter(mu_init, sigma_init, A, B, sigma_states, C, D, sigma_obs, inputs, data)
    
#new_lls, new_filtered_mus, new_filtered_sigmas, expected = E_step(mu_init, sigma_init, A, B, sigma_states, C, D, sigma_obs, inputs, data)   
  

In [6]:
def kalman_convergence(subject, n_dim_state, n_time_points):
    #parameter: subject, state dimensions, # of time points 
    
    x = np.array(group2[subject]['processed']['deltaFOverF_bc_detr'][:]);
    
    x = x[:,0:n_time_points]
    
    x = x.T
    
    
    T = n_time_points #T = 3000
    D = n_dim_state #D = 10
    N = x.shape[1] #N = 109
    
    model = DefaultLDS(N, D, 0) #D_obs, D_latent, D_input
    model.add_data(x) #add this

    
    def update(model):
        model.EM_step()
        return model.log_likelihood()
    
    
    
    converged = False
    
    tol = 1e-8
    
    #lls = [update(model) for _ in progprint_xrange(50)]
    
    i = 0
    while converged == False:
        
        print(i)
        
        log_likelihood = update(model)
        
        LL.append(log_likelihood)

        LLold = LL[i]

        if i<=2:

            LLbase = LL[i];

        elif ((LL[i]-LLbase) < (1+tol)*(LLold-LLbase)):

            converged = True
       
        i = i + 1
        
    #plt.plot(LL)
    
    print(LL)
        
    return i - 1

In [7]:
kalman_convergence('N1', 10, 3000)

ImportError: dlopen(/Users/anastasia/anaconda3/lib/python3.6/site-packages/pypolyagamma/pypolyagamma.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libgsl.23.dylib
  Referenced from: /Users/anastasia/anaconda3/lib/python3.6/site-packages/pypolyagamma/pypolyagamma.cpython-36m-darwin.so
  Reason: image not found